In [40]:
# import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

In [2]:
# load and read dataset
dataset = pd.read_csv('Train.xls')
dataset.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
# shape of dataset
dataset.shape

(8068, 11)

In [4]:
# find the null values
dataset.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [5]:
# find other information
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [6]:
dataset['Ever_Married'].value_counts()

Ever_Married
Yes    4643
No     3285
Name: count, dtype: int64

In [7]:
# as yes are more so put yes in the null values in Ever_Married column
dataset['Ever_Married'] = dataset['Ever_Married'].fillna('Yes')
dataset['Ever_Married']

0        No
1       Yes
2       Yes
3       Yes
4       Yes
       ... 
8063     No
8064     No
8065     No
8066     No
8067    Yes
Name: Ever_Married, Length: 8068, dtype: object

In [8]:
dataset.Ever_Married.value_counts()

Ever_Married
Yes    4783
No     3285
Name: count, dtype: int64

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     8068 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [10]:
# find unique values in graduated column
dataset['Graduated'].value_counts()

Graduated
Yes    4968
No     3022
Name: count, dtype: int64

In [11]:
# as mode in graduated column is yes, so lets fill the null values with yes
dataset.Graduated = dataset.Graduated.fillna('Yes')
dataset['Graduated']

0        No
1       Yes
2       Yes
3       Yes
4       Yes
       ... 
8063     No
8064     No
8065    Yes
8066    Yes
8067    Yes
Name: Graduated, Length: 8068, dtype: object

In [12]:
# now look for unique values in Profession column
dataset.Profession.value_counts()

Profession
Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: count, dtype: int64

In [13]:
# find number of null values in profession column
dataset.Profession.isnull().sum()

124

In [14]:
# lets fill some null values with Artist and others with Healthcare

# create a mask of all the null values in profession column
# prof_mask = dataset.Profession.isnull()
art = 'Artist'
health = 'Healthcare'

# now fill some values with Artist
dataset.loc[dataset['Profession'].isnull() & (dataset.index % 2 == 0), 'Profession'] = art
dataset.loc[dataset['Profession'].isnull() & (dataset.index % 2 != 0), 'Profession'] = health

dataset.Profession

0          Healthcare
1            Engineer
2            Engineer
3              Lawyer
4       Entertainment
            ...      
8063       Healthcare
8064        Executive
8065       Healthcare
8066       Healthcare
8067        Executive
Name: Profession, Length: 8068, dtype: object

In [15]:
dataset.Profession.isnull().sum()

0

In [16]:
dataset.Profession.value_counts()

Profession
Artist           2568
Healthcare       1404
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: count, dtype: int64

In [17]:
dataset.isnull().sum()

ID                   0
Gender               0
Ever_Married         0
Age                  0
Graduated            0
Profession           0
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [18]:
# now find unique values in Work_Experience and fill them
dataset.Work_Experience.value_counts()

Work_Experience
1.0     2354
0.0     2318
9.0      474
8.0      463
2.0      286
3.0      255
4.0      253
6.0      204
7.0      196
5.0      194
10.0      53
11.0      50
12.0      48
13.0      46
14.0      45
Name: count, dtype: int64

In [19]:
# fill some values in Work_Experience with 1.0, some with 0.0, some with 9.0, and remaining with median, mean
median = dataset['Work_Experience'].median()
mean = round(dataset.Work_Experience.mean(), 1)

dataset.loc[dataset.Work_Experience.isnull() & (dataset.index % 7 == 0), 'Work_Experience'] = 1.0

dataset.loc[dataset.Work_Experience.isnull() & (dataset.index % 5 == 0), 'Work_Experience'] = 0.0

dataset.loc[dataset.Work_Experience.isnull() & (dataset.index % 3 == 0), 'Work_Experience'] = 9.0

dataset.loc[dataset.Work_Experience.isnull() & (dataset.index % 2 == 0), 'Work_Experience'] = 8.0

dataset.loc[dataset.Work_Experience.isnull() & (dataset.index % 2 != 0), 'Work_Experience'] = 2.0

dataset.Work_Experience.isnull().sum()

0

In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     8068 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        8068 non-null   object 
 5   Profession       8068 non-null   object 
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [21]:
# find missing values in family size column and deal with them
dataset.Family_Size.isnull().sum()

335

In [22]:
# deal with the missing values
dataset['Family_Size'].value_counts()

Family_Size
2.0    2390
3.0    1497
1.0    1453
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: count, dtype: int64

In [26]:
# now populate the null values accordingly
mode = dataset['Family_Size'].mode()
mean = dataset['Family_Size'].mean()
median = dataset['Family_Size'].median()
median

3.0

In [27]:
dataset.loc[dataset.Family_Size.isnull() & (dataset.index % 5 == 0), 'Family_Size'] = 2.0
dataset.loc[dataset['Family_Size'].isnull() & (dataset.index % 3 == 0), 'Family_Size'] = 3.0
dataset.loc[dataset['Family_Size'].isnull() & (dataset.index % 2 == 0), 'Family_Size'] = mean
dataset.loc[dataset.Family_Size.isnull() & (dataset.index % 2 != 0), 'Family_Size'] = 1.0

In [28]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     8068 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        8068 non-null   object 
 5   Profession       8068 non-null   object 
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      8068 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [31]:
# no of missing values in Var_1 column
dataset.Var_1.isnull().sum()

76

In [30]:
# find all unique values in Var_1
dataset['Var_1'].value_counts()

Var_1
Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: count, dtype: int64

In [32]:
# adjust all the missing values in Var_1 column Cat_4
dataset.loc[dataset.Var_1.isnull() & (dataset.index % 2 == 0), 'Var_1'] = 'Cat_6'
dataset.loc[dataset['Var_1'].isnull() & (dataset.index % 2 != 0), 'Var_1'] = 'Cat_4'

In [33]:
dataset.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [34]:
# now display dataset
dataset.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,2.0,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,8.0,High,6.0,Cat_6,A


In [35]:
# we do not need ID column so remove it
dataset = dataset.drop(columns = ['ID'], axis = 1)
dataset.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,Female,Yes,38,Yes,Engineer,2.0,Average,3.0,Cat_4,A
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,Female,Yes,40,Yes,Entertainment,8.0,High,6.0,Cat_6,A


In [36]:
# find datatypes of all the columns in dataset
dataset.dtypes

Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Var_1               object
Segmentation        object
dtype: object

In [38]:
# as there are two unique values in some columns, use labelencoder from sklearn to encode them
encoder = LabelEncoder()

# encode for Gender
dataset['Gender'] = encoder.fit_transform(dataset.Gender)

# encode for Ever_Married
dataset['Ever_Married'] = encoder.fit_transform(dataset['Ever_Married'])

# encode Graduated
dataset['Graduated'] = encoder.fit_transform(dataset['Graduated'])

dataset.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,1,0,22,0,Healthcare,1.0,Low,4.0,Cat_4,D
1,0,1,38,1,Engineer,2.0,Average,3.0,Cat_4,A
2,0,1,67,1,Engineer,1.0,Low,1.0,Cat_6,B
3,1,1,67,1,Lawyer,0.0,High,2.0,Cat_6,B
4,0,1,40,1,Entertainment,8.0,High,6.0,Cat_6,A


In [39]:
# Spending_Score has ordinal data, so apply ordinal encoding
dataset.Spending_Score.value_counts()

Spending_Score
Low        4878
Average    1974
High       1216
Name: count, dtype: int64

In [45]:
# apply ordinalencoding
ordinal_encode = OrdinalEncoder(categories = [['Low', 'Average', 'High']]) 

dataset['Spending_Score'] = ordinal_encode.fit_transform(dataset[['Spending_Score']])
dataset.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,1,0,22,0,Healthcare,1.0,0.0,4.0,Cat_4,D
1,0,1,38,1,Engineer,2.0,1.0,3.0,Cat_4,A
2,0,1,67,1,Engineer,1.0,0.0,1.0,Cat_6,B
3,1,1,67,1,Lawyer,0.0,2.0,2.0,Cat_6,B
4,0,1,40,1,Entertainment,8.0,2.0,6.0,Cat_6,A


In [ ]:
# apply one hot encoding
dataset[]